In [1]:
import pandas as pd
import numpy as np

In [ ]:
consumption_col = 'AEP_MW'

df = pd.read_csv('./rsc/AEP_hourly.csv')
df['Date'] = pd.to_datetime(df['Datetime'])
df = df.drop(columns=['Datetime'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121273 entries, 0 to 121272
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   AEP_MW  121273 non-null  float64       
 1   Date    121273 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.9 MB
None


# Classificazione rispetto alla media GLOBALE

In [3]:
global_mean = df[consumption_col].mean()
df['classification_global'] = np.where(df[consumption_col] >= global_mean, 'Alto Consumo', 'Basso Consumo')
print(df)

         AEP_MW                Date classification_global
0       13478.0 2004-12-31 01:00:00         Basso Consumo
1       12865.0 2004-12-31 02:00:00         Basso Consumo
2       12577.0 2004-12-31 03:00:00         Basso Consumo
3       12517.0 2004-12-31 04:00:00         Basso Consumo
4       12670.0 2004-12-31 05:00:00         Basso Consumo
...         ...                 ...                   ...
121268  21089.0 2018-01-01 20:00:00          Alto Consumo
121269  20999.0 2018-01-01 21:00:00          Alto Consumo
121270  20820.0 2018-01-01 22:00:00          Alto Consumo
121271  20415.0 2018-01-01 23:00:00          Alto Consumo
121272  19993.0 2018-01-02 00:00:00          Alto Consumo

[121273 rows x 3 columns]


# Classificazione rispetto alla media Settimanale

In [4]:
df['week'] = df['Date'].dt.isocalendar().week
df['year'] = df['Date'].dt.year
df['year_week'] = df['year'].astype(str) + '_' + df['week'].astype(str)
weekly_means = df.groupby('year_week')[consumption_col].mean()
df['weekly_mean'] = df['year_week'].map(weekly_means)
df['classification_weekly'] = np.where(df[consumption_col] >= df['weekly_mean'], 'Alto Consumo', 'Basso Consumo')
print(df)

         AEP_MW                Date classification_global  week  year  \
0       13478.0 2004-12-31 01:00:00         Basso Consumo    53  2004   
1       12865.0 2004-12-31 02:00:00         Basso Consumo    53  2004   
2       12577.0 2004-12-31 03:00:00         Basso Consumo    53  2004   
3       12517.0 2004-12-31 04:00:00         Basso Consumo    53  2004   
4       12670.0 2004-12-31 05:00:00         Basso Consumo    53  2004   
...         ...                 ...                   ...   ...   ...   
121268  21089.0 2018-01-01 20:00:00          Alto Consumo     1  2018   
121269  20999.0 2018-01-01 21:00:00          Alto Consumo     1  2018   
121270  20820.0 2018-01-01 22:00:00          Alto Consumo     1  2018   
121271  20415.0 2018-01-01 23:00:00          Alto Consumo     1  2018   
121272  19993.0 2018-01-02 00:00:00          Alto Consumo     1  2018   

       year_week   weekly_mean classification_weekly  
0        2004_53  16419.700000         Basso Consumo  
1        2004

# Classificazione rispetto alla media GIORNALIERA

In [5]:
daily_means = df.groupby('Date')[consumption_col].mean()
df['daily_mean'] = df['Date'].map(daily_means)
df['classification_daily'] = np.where(df[consumption_col] >= df['daily_mean'], 'Alto Consumo', 'Basso Consumo')
df = df.drop(columns=['daily_mean'])
print(df)

         AEP_MW                Date classification_global  week  year  \
0       13478.0 2004-12-31 01:00:00         Basso Consumo    53  2004   
1       12865.0 2004-12-31 02:00:00         Basso Consumo    53  2004   
2       12577.0 2004-12-31 03:00:00         Basso Consumo    53  2004   
3       12517.0 2004-12-31 04:00:00         Basso Consumo    53  2004   
4       12670.0 2004-12-31 05:00:00         Basso Consumo    53  2004   
...         ...                 ...                   ...   ...   ...   
121268  21089.0 2018-01-01 20:00:00          Alto Consumo     1  2018   
121269  20999.0 2018-01-01 21:00:00          Alto Consumo     1  2018   
121270  20820.0 2018-01-01 22:00:00          Alto Consumo     1  2018   
121271  20415.0 2018-01-01 23:00:00          Alto Consumo     1  2018   
121272  19993.0 2018-01-02 00:00:00          Alto Consumo     1  2018   

       year_week   weekly_mean classification_weekly classification_daily  
0        2004_53  16419.700000         Basso Co